# Benchmarks

Against some alternatives.

## Data

UCI Credit card dataset with 30k rows and 23 features.

In [1]:
import pandas as pd
from skorecard.datasets import load_credit_card
from sklearn.model_selection import train_test_split

data = load_credit_card(as_frame=True)
print(f"data shape: {data.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['y'], axis=1),
    data['y'], 
    test_size=0.25, 
    random_state=42
)

# different format also
df_train = X_train.copy()
df_train['y'] = y_train

data shape: (30000, 24)


## Experiment setup

In [2]:
from sklearn.metrics import roc_auc_score

def report_auc(clf, X_train, y_train, X_test, y_test):
    proba_train = clf.predict_proba(X_train)[:,1]
    proba_test = clf.predict_proba(X_test)[:,1]

    auc_train = round(roc_auc_score(y_train, proba_train),4) 
    auc_test = round(roc_auc_score(y_test, proba_test),4)

    return auc_train, auc_test


In [3]:
from memo import memlist, time_taken

data = []

@memlist(data=data)
@time_taken()
def fit_eval_record(clf, name):
    
    if name == "optbinning.Scorecard":
        clf.fit(df_train)
    else:
        clf.fit(X_train, y_train)

    auc_train, auc_test = report_auc(clf, X_train, y_train, X_test, y_test)

    return {'auc_train': auc_train, 'auc_test': auc_test}

## Baseline

In [4]:
from skorecard import Skorecard

scorecard = Skorecard()
fit_eval_record(scorecard, name="skorecard.Scorecard")

{'auc_train': 0.7737, 'auc_test': 0.7654, 'time_taken': 9.22}

## Optbinning

http://gnpalencia.org/optbinning/

In [6]:
from optbinning import BinningProcess
from optbinning import Scorecard
from sklearn.linear_model import LogisticRegression
import pandas as pd

selection_criteria = {
    "iv": {"min": 0.02, "max": 1},
    "quality_score": {"min": 0.01}
}
binning_process = BinningProcess(variable_names = list(X_train.columns), selection_criteria=selection_criteria)

estimator = LogisticRegression(solver="lbfgs")



opt_scorecard = Scorecard(target="y", binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 300, "max": 850})


fit_eval_record(opt_scorecard, name="optbinning.Scorecard")

{'auc_train': 0.7719, 'auc_test': 0.7628, 'time_taken': 1.76}

## Basic LR

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression(random_state=42)
)

fit_eval_record(pipe, name="sklearn.LogisticRegression")

{'auc_train': 0.724, 'auc_test': 0.7232, 'time_taken': 0.17}

## LightGBM model

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [13]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(random_state=42, max_depth=10, learning_rate=0.01)

fit_eval_record(clf, name="LightGBM")


{'auc_train': 0.8038, 'auc_test': 0.7778, 'time_taken': 0.24}

## Results

In [10]:
pd.DataFrame(data).sort_values('auc_test', ascending=False)

,name,auc_train,auc_test,time_taken
0,skorecard.Scorecard,0.7737,0.7654,9.22
1,optbinning.Scorecard,0.7719,0.7628,1.76
2,sklearn.LogisticRegression,0.7240,0.7232,0.17
3,LightGBM,0.7240,0.7232,0.09
